In [1]:
import requests
import pandas as pd

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

In [2]:
ALL_EVENTS_URL = "https://www.roh.org.uk/api/events"
r = requests.get(ALL_EVENTS_URL)
data = r.json()

In [3]:
events_df = pd.DataFrame(data["data"])
included_df = pd.DataFrame(data["included"])

events_attrs = events_df.attributes.apply(pd.Series)
events_attrs.drop(
    columns=[
        "description",
        "dateFieldOverride",
        "imageResult",
        "imageTray",
        "productionPageUrl",
        "helpInformation",
    ],
    inplace=True,
)
events_rels = events_df.relationships.apply(pd.Series)
events_df = pd.concat([events_df, events_attrs, events_rels], axis=1)
events_df.query("isCancelled != True", inplace=True)
events_df.drop(
    columns=[
        "attributes",
        "relationships",
        "isCancelled",
        "ctaBehaviour",
        "cinemaBroadcastLink",
    ],
    inplace=True,
)
# remove events with no listed performances
events_df.query("performances not in [[]]", inplace=True)
events_df.reset_index(drop=True, inplace=True)

In [4]:
locations_df = included_df.query("type == 'locations'").drop(
    columns=["type", "relationships"]
)
locations_attrs = locations_df.attributes.apply(pd.Series)
locations_df = pd.concat([locations_df, locations_attrs], axis=1)
locations_df.drop(columns=["attributes"], inplace=True)
locations_df.reset_index(drop=True, inplace=True)
locations_df.rename(columns={"id": "locationId", "title": "location"}, inplace=True)

In [5]:
events_df = events_df.explode("performances", ignore_index=True)
events_df.locations = events_df.locations.apply(lambda x: x["data"])
events_df = events_df.explode("locations", ignore_index=True)

In [6]:
events_df.locations = events_df.locations.apply(lambda x: x["id"])
events_df.rename(columns={"locations": "locationId"}, inplace=True)
events_df = events_df.merge(locations_df, on="locationId", how="left")

In [7]:
performances_df = events_df.performances.apply(pd.Series)
performances_df = performances_df.assign(
    timestamp=pd.to_datetime(performances_df.date, utc=True)
)
# Adjust the timestamp to account for UK timezone
performances_df.timestamp = performances_df.timestamp.dt.tz_convert("Europe/London")
events_df = pd.concat([events_df, performances_df], axis=1)
events_df.drop(columns=["performances", "date"], inplace=True)

In [8]:
today = pd.Timestamp.today(tz="Europe/London") - pd.Timedelta(hours=1)
tomorrow = today + pd.Timedelta(days=1)
events_df = events_df.query("timestamp > @today")
events_df.sort_values(by=["timestamp"], inplace=True)

In [9]:
events_df["date"] = events_df.timestamp.dt.date
events_df["time"] = events_df.timestamp.dt.time
events_df["day"] = events_df.timestamp.dt.day_name()

In [10]:
print("Today's and tomorrow's events:")
today_tomorrow_events_df = events_df.query("date in [@today.date(), @tomorrow.date()]")
today_tomorrow_events_df.loc[:, ["id", "title", "location", "date", "day", "time"]]

Today's and tomorrow's events:


,id,title,location,date,day,time
18,8513,Friends Rehearsals,Main Stage,2024-02-02,Friday,11:30:00
179,33825,La bohème,Main Stage,2024-02-02,Friday,19:30:00
339,2581,Manon,Main Stage,2024-02-03,Saturday,13:00:00
340,2581,Manon,Main Stage,2024-02-03,Saturday,19:00:00


In [11]:
today_tomorrow_events_df

,type,id,sourceType,title,slug,startTime,endTime,isHiddenFromTicketsAndEvents,tags,bookingSeasons,locationId,performanceTimes,onSaleDates,runs,festival,location,performanceType,timestamp,date,time,day
18,event,8513,event-detail,Friends Rehearsals,friends-rehearsals,None,None,False,"{'data': [{'type': 'tags', 'id': '1132'}, {'ty...","{'data': [{'id': '977', 'type': 'bookingSeason...",2,"{'data': [{'id': '936', 'type': 'performanceTi...",{'data': [{'id': 'd166a0ddb23265d2ebc15e6eb99e...,"{'data': [{'id': '54563', 'type': 'runs'}, {'i...",{'data': None},Main Stage,focg,2024-02-02 11:30:00+00:00,2024-02-02,11:30:00,Friday
179,event,33825,event-detail,La bohème,la-boheme-by-richard-jones,None,None,False,"{'data': [{'type': 'tags', 'id': '830'}, {'typ...","{'data': [{'id': '978', 'type': 'bookingSeason...",2,"{'data': [{'id': '936', 'type': 'performanceTi...",{'data': [{'id': '108c7536e57f0e464773eabb2f3b...,"{'data': [{'id': '54605', 'type': 'runs'}]}",{'data': None},Main Stage,standard,2024-02-02 19:30:00+00:00,2024-02-02,19:30:00,Friday
339,event,2581,event-detail,Manon,manon-by-kenneth-macmillan,None,None,False,"{'data': [{'type': 'tags', 'id': '922'}, {'typ...","{'data': [{'id': '978', 'type': 'bookingSeason...",2,"{'data': [{'id': '936', 'type': 'performanceTi...",{'data': [{'id': '108c7536e57f0e464773eabb2f3b...,"{'data': [{'id': '54607', 'type': 'runs'}]}",{'data': None},Main Stage,standard,2024-02-03 13:00:00+00:00,2024-02-03,13:00:00,Saturday
340,event,2581,event-detail,Manon,manon-by-kenneth-macmillan,None,None,False,"{'data': [{'type': 'tags', 'id': '922'}, {'typ...","{'data': [{'id': '978', 'type': 'bookingSeason...",2,"{'data': [{'id': '936', 'type': 'performanceTi...",{'data': [{'id': '108c7536e57f0e464773eabb2f3b...,"{'data': [{'id': '54607', 'type': 'runs'}]}",{'data': None},Main Stage,standard,2024-02-03 19:00:00+00:00,2024-02-03,19:00:00,Saturday


In [14]:
import plotly.express as px

events_df["Size"] = 1  # set the size of the dots
events_df["url"] = events_df.slug.apply(lambda x: f"https://www.roh.org.uk/{x}-details")

events_df_sub = events_df.query("location == 'Main Stage'")
fig = px.scatter(
    events_df_sub,
    custom_data=["title", "url"],
    x="date",
    y="time",
    color="title",
    size="Size",
    size_max=7,
    title="Royal Opera House Events",
    template="simple_white",
    hover_name="url",
)
# Keep first 5 characters of the y axis marks
fig.update_yaxes(
    categoryorder="category ascending",
    showgrid=True,
    tickvals=events_df_sub.time.unique(),
    ticktext=[str(x)[:5] for x in events_df_sub.time.unique()],
    title="",
)
fig.update_xaxes(
    title="",
    showgrid=True,
    gridwidth=1,
    gridcolor="LightGray",
)
fig.update_layout(
    width=1700,
    height=500,
    margin=dict(l=0, r=0, b=0, t=40, pad=0),
)
fig.update_traces(
    marker=dict(line=dict(width=1, color="DarkSlateGrey")),
    selector=dict(mode="markers"),
)
fig.update_layout(
    hoverlabel=dict(
        font_size=16,
        font_family="Gotham",
        font_color="White",
        bgcolor="#C7102E",
    ),
    legend=dict(
        title="Title",
        title_font=dict(size=15, family="Gotham"),
        font=dict(size=13, family="Gotham"),
    ),
    title=None
)
fig.layout.font.family = "Gotham"
# Remove Size from the hover data
fig.update_traces(
    hovertemplate="<br>".join(
        [
            "%{customdata[0]}",
            "%{x}",
            "%{y}",
        ],
    ) + "<extra></extra>",
)
fig.show()

In [15]:
fig.write_image("ROH_events.png", scale=3)